In [11]:
import http.server
import datetime
import socketserver
import threading
import subprocess
import time
import json
import requests

In [12]:
long_access_token = 'EAASGrSvhX68BOyIv0FPDLYTYEjM2gGyrWvAEOVfSR9mP6HHKQaiOVYxg9PWsZBdxhej5rV4TBQIh1ooj5DocKM7yObTAjOIjBAmrS4DZBRv6BDLGDqUCOkskQP33YTyy961su1FkSeAX4ONZCRHv13W05AI541ez8Qs2RQt8KE1ZC4m9wYkGKXr5'

In [13]:
ig_user_id = "17841471793837113"

In [14]:
PORT = 8000

# Keep incrementing the port until an available one is found
while True:
    try:
        Handler = http.server.SimpleHTTPRequestHandler
        httpd = socketserver.TCPServer(("", PORT), Handler)
        break  # Exit loop if no error occurs
    except OSError:
        print(f"Port {PORT} is in use. Trying port {PORT + 1}...")
        PORT += 1 

Port 8000 is in use. Trying port 8001...


In [15]:
ngrok = subprocess.Popen([r"C:\Users\Mohnish\Desktop\retail customer analysis\api\output\ngrok.exe", "http", str(PORT)], stdout=subprocess.PIPE)

time.sleep(2)

In [16]:
resp = requests.get("http://localhost:4040/api/tunnels")
public_url = resp.json()["tunnels"][0]["public_url"]

image_url = public_url + "/output/reel.mp4"
print(image_url)

https://f56d-2401-4900-6341-4159-d9dd-25a8-d6f5-a886.ngrok-free.app/output/reel.mp4


In [17]:
caption = '''
Chinna Porur Yejamaan
'''
print(caption)


Chinna Porur Yejamaan



In [18]:
url = f"https://graph.facebook.com/v17.0/{ig_user_id}/media"

payload = {
    "media_type": "REELS",
    "video_url" : image_url,
    "caption" : caption,
    "share_to_feed" : "false",
    "access_token" : long_access_token
}

response = requests.post(url, params=payload)
data = response.json()
print(json.dumps(data, indent=4))
creation_id = data["id"]
print(f"creation_id : {creation_id}")

{
    "id": "18073677880777315"
}
creation_id : 18073677880777315


In [19]:
url = f"https://graph.facebook.com/v17.0/{ig_user_id}/media_publish"
payload = {
    "creation_id": creation_id,
    "access_token": long_access_token
}

max_retries = 5  # Maximum number of attempts
attempt = 0

while attempt < max_retries:
    try:
        time.sleep(10)
        response = requests.post(url, params=payload)
        data = response.json()

        if "id" in data:  # Success condition
            media_id = data["id"]
            print(f"Media ID: {media_id}")
            break  # Exit loop if successful

        print(f"Attempt {attempt + 1} failed: {json.dumps(data, indent=4)}")

    except requests.exceptions.RequestException as e:
        print(f"Attempt {attempt + 1} failed due to a request error: {e}")

    attempt += 1
    time.sleep(2)  # Wait before retrying

if attempt == max_retries:
    print("Max retries reached. Could not publish media.")

Media ID: 18022492751646358


In [20]:
url = f"https://graph.facebook.com/v21.0/{media_id}"
payload={
    "fields":"comments_count, like_count, media_url, permalink",
    "access_token":long_access_token
}

response = requests.get(url, params=payload)
data = response.json()
print(json.dumps(data, indent=4))
permalink = data["permalink"]
print(f"permalink : {permalink}")

{
    "comments_count": 0,
    "like_count": 0,
    "media_url": "https://instagram.fmaa5-1.fna.fbcdn.net/o1/v/t16/f2/m86/AQNYXGVda1MvWvr_jSI-uOFzFs9at_y7Sel5VVeIdNutoA5ihezRH8ZuZY30BKNpElG8dmsEf0wgLmk6-o_Q4sF2VKvEsOcVv4DTj1I.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192b2RfdXJsZ2VuLmNsaXBzLnVua25vd24tQzMuNzIwLmRhc2hfYmFzZWxpbmVfMV92MSJ9&_nc_ht=instagram.fmaa5-1.fna.fbcdn.net&_nc_cat=102&vs=1120717546035472_3736039603&_nc_vs=HBkcFQIYUmlnX3hwdl9yZWVsc19wZXJtYW5lbnRfc3JfcHJvZC83MTRBMzdDNTYwNUVCMkRBRkUzOUYwMDk5NUUzNUU4RF92aWRlb19kYXNoaW5pdC5tcDQVAALIAQAoABgAGwGIB3VzZV9vaWwBMRUAACaGyN7Y%2FOSSQBUCKAJDMywXQCgAAAAAAAAYEmRhc2hfYmFzZWxpbmVfMV92MREAdQAA&ccb=9-4&oh=00_AYBXMX73Bf2KwYsPZX0JXP6C-bXhM0CYTZFaiVqm2RUNeA&oe=67A289E3&_nc_sid=1d576d",
    "permalink": "https://www.instagram.com/reel/DFm3c7Dp2W6/",
    "id": "18022492751646358"
}
permalink : https://www.instagram.com/reel/DFm3c7Dp2W6/
